In [16]:
import sys
!{sys.executable} -m pip install pandas requests lxml html5lib

    100% |████████████████████████████████| 122kB 4.2MB/s ta 0:00:01


In [18]:
import requests
import re
import numpy as np
import pandas as pd
import html5lib

moscow_postcodes_link='http://mosopen.ru/streets/post_codes_list'
raw_moscow_postcodes_page=requests.get(moscow_postcodes_link)
page=raw_moscow_postcodes_page.text

In [21]:
page_table_start = page.find('<table class="table_list">')
page_table_end = page.find('</table>', page_table_start)
print(page_table_start, page_table_end)

17716 66404


In [22]:
page_table = page[page_table_start : page_table_end]

In [97]:
df = pd.read_html(page_table, header =0)[0]
# Rename columns
df.columns = ['city_code', 'postal_code']
# Transform city_code to string
df['city_code']= df['city_code'].astype(str)

In [78]:
postcodes = df['postal_code'].str.split(',', expand=True).stack().str.strip().reset_index(level=1, drop=True)
postcodes = postcodes.rename('postal_code')

In [98]:
df = df.drop(['postal_code'], axis=1)

In [99]:
df = pd.concat([df, postcodes], axis=1)

In [100]:
df['postal_code'] = df['city_code'] + df['postal_code']

In [101]:
df = df.drop(['city_code'], axis=1)

,postal_code
0,101000
1,103070
1,103132
1,103274
2,105005
2,105037
2,105043
2,105062
2,105064
2,105066


In [ ]:
# Get first table from page

df = df[df['Borough'] != 'Not assigned']
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(", ".join).reset_index()
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'], df['Neighbourhood'])